##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Context Caching Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Caching.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

This notebook introduces context caching with the Gemini API and provides examples of interacting with the Apollo 11 transcript using the Python SDK. For a more comprehensive look, check out [the caching guide](https://ai.google.dev/gemini-api/docs/caching?lang=python).

### Install dependencies

In [1]:
%pip install -U -q "google-genai>=1.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.3 MB/s eta 0:00:00


In [2]:
from google import genai
from google.genai import types

### Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Upload a file

A common pattern with the Gemini API is to ask a number of questions of the same document. Context caching is designed to assist with this case, and can be more efficient by avoiding the need to pass the same tokens through the model for each new request.

This example will be based on the transcript from the Apollo 11 mission.

Start by downloading that transcript.

In [4]:
!wget -q https://storage.googleapis.com/generativeai-downloads/data/a11.txt
!head a11.txt

INTRODUCTION

This is the transcription of the Technical Air-to-Ground Voice Transmission (GOSS NET 1) from the Apollo 11 mission.

Communicators in the text may be identified according to the following list.

Spacecraft:
CDR	Commander	Neil A. Armstrong
CMP	Command module pilot   	Michael Collins
LMP	Lunar module pilot	Edwin E. ALdrin, Jr.


Now upload the transcript using the [File API](../quickstarts/File_API.ipynb).

In [6]:
document = client.files.upload(file="a11.txt")

## Cache the prompt

Next create a [`CachedContent`](https://ai.google.dev/api/python/google/generativeai/protos/CachedContent) object specifying the prompt you want to use, including the file and other fields you wish to cache. In this example the [`system_instruction`](../quickstarts/System_instructions.ipynb) has been set, and the document was provided in the prompt.

In [11]:
# Note that caching requires a frozen model, e.g. one with a `-001` suffix.
MODEL_ID="gemini-1.5-flash-002"  # @param ["gemini-1.5-flash-002","gemini-1.5-pro-002","gemini-1.5-flash-8b-latest"] {"allow-input":true, isTemplate: true}

apollo_cache = client.caches.create(
    model=MODEL_ID,
    config={
        'contents': [document],
        'system_instruction': 'You are an expert at analyzing transcripts.',
    },
)

apollo_cache

CachedContent(name='cachedContents/zqizlpe6fwr4', display_name='', model='models/gemini-1.5-flash-002', create_time=datetime.datetime(2025, 3, 24, 15, 18, 46, 527679, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 3, 24, 15, 18, 46, 527679, tzinfo=TzInfo(UTC)), expire_time=datetime.datetime(2025, 3, 24, 16, 18, 45, 932839, tzinfo=TzInfo(UTC)), usage_metadata=CachedContentUsageMetadata(audio_duration_seconds=None, image_count=None, text_count=None, total_token_count=323384, video_duration_seconds=None))

## Manage the cache expiry

Once you have a `CachedContent` object, you can update the expiry time to keep it alive while you need it.

In [12]:
client.caches.update(
    name=apollo_cache.name,
    config=types.UpdateCachedContentConfig(ttl="7200s")  # 2 hours in seconds
)
apollo_cache = client.caches.get(name=apollo_cache.name) # Get the updated cache
apollo_cache

CachedContent(name='cachedContents/zqizlpe6fwr4', display_name='', model='models/gemini-1.5-flash-002', create_time=datetime.datetime(2025, 3, 24, 15, 18, 46, 527679, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 3, 24, 15, 19, 35, 689164, tzinfo=TzInfo(UTC)), expire_time=datetime.datetime(2025, 3, 24, 17, 19, 35, 677457, tzinfo=TzInfo(UTC)), usage_metadata=CachedContentUsageMetadata(audio_duration_seconds=None, image_count=None, text_count=None, total_token_count=323384, video_duration_seconds=None))

## Use the cache for generation

As the `CachedContent` object refers to a specific model and parameters, you must create a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) using [`from_cached_content`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#from_cached_content). Then, generate content as you would with a directly instantiated model object.

In [14]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents='Find a lighthearted moment from this transcript',
    config=types.GenerateContentConfig(
        cached_content=apollo_cache.name,
    )
)

print(response.text)

Several lighthearted moments punctuate the otherwise technical Apollo 11 transcript.  Here are a couple, with slightly different tones:

* **The "cup of coffee" bet:**  On page 4,  CMP quips about not owing Glenn Parker a cup of coffee after a successful alignment. This is a playful jab at a colleague and suggests a friendly, informal relationship among the crew.

* **The living room comment:** On page 2, CDR Armstrong comments that  CC sounds like he's "sitting in your living room," which is a humorous contrast to the intense, high-stakes environment of a space launch. The appreciative response from CC makes it a shared moment of levity.


While there are other instances of informal language and camaraderie, these two examples stand out as distinct lighthearted moments.  The humor in these exchanges relies on the contrast between the gravity of the situation and the casual tone of the conversation.



You can inspect token usage through `usage_metadata`. Note that the cached prompt tokens are included in `prompt_token_count`, but excluded from the `total_token_count`.

In [15]:
response.usage_metadata

GenerateContentResponseUsageMetadata(cached_content_token_count=323384, candidates_token_count=191, prompt_token_count=323392, total_token_count=323583)

You can ask new questions of the model, and the cache is reused.

In [16]:
chat = client.chats.create(
  model = MODEL_ID,
  config = {"cached_content": apollo_cache.name}
)

response = chat.send_message(message="Give me a quote from the most important part of the transcript.")
print(response.text)

The most important part of the transcript is the moment of the moon landing.  The quote is:

**"Houston, Tranquility Base here. The Eagle has landed."**



In [20]:
response = chat.send_message(
    message="What was recounted after that?",
    config = {"cached_content": apollo_cache.name}
)
print(response.text)

My apologies, I seem to have gotten stuck on the initial post-landing sequence.  The transcript continues *after* the moon landing with extensive details of the Apollo 11 mission.  Here's a summary of what is recounted *after* the initial post-landing sequence:

The transcript details the many hours of activity following the moon landing, covering:

* **EVA (Extravehicular Activity):**  A lengthy section details the astronauts' activities on the lunar surface, including their movements, descriptions of the terrain, collection of samples (rock, soil, etc.), deployment of the American flag, and use of the scientific equipment.  The communication highlights their observations of the lunar surface, the ease of movement in low gravity, and the appearance of the lunar landscape.

* **Communications Difficulties:** There were recurring instances of communication dropouts due to antenna alignment issues, interference, and equipment problems.  The astronauts and ground control spent considerabl

In [18]:
response.usage_metadata

GenerateContentResponseUsageMetadata(cached_content_token_count=323384, candidates_token_count=448, prompt_token_count=323441, total_token_count=323889)

As you can see, among the 323455 tokens, 323383 were cached (and thus less expensive) and only 236 were from the prompt.

Since the cached tokens are cheaper than the normal ones, it means this prompt was 75% cheaper that if you had not used caching. Check the [pricing here](https://ai.google.dev/pricing) for the up-to-date discount on cached tokens.

## Delete the cache

The cache has a small recurring storage cost (cf. [pricing](https://ai.google.dev/pricing)) so by default it is only saved for an hour. In this case you even set it up for a shorter amont of time (using `"ttl"`) of 2h.

Still, if you don't need you cache anymore, it is good practice to delete it proactively.

In [21]:
print(apollo_cache.name)
client.caches.delete(name=apollo_cache.name)

cachedContents/zqizlpe6fwr4


DeleteCachedContentResponse()

## Next Steps
### Useful API references:

If you want to know more about the caching API, you can check the full [API specifications](https://ai.google.dev/api/rest/v1beta/cachedContents) and the [caching documentation](https://ai.google.dev/gemini-api/docs/caching).

### Continue your discovery of the Gemini API

Check the File API notebook to know more about that API. The [vision capabilities](../quickstarts/Video.ipynb) of the Gemini API are a good reason to use the File API and the caching.
The Gemini API also has configurable [safety settings](../quickstarts/Safety.ipynb) that you might have to customize when dealing with big files.
